In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"
# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres. 
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-02 15:29:15--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.94MB/s    in 0.2s    

2022-04-02 15:29:15 (4.94 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Cleaned-Wine-Data").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Load combined weather data into Spark DataFrame
from pyspark import SparkFiles
url = "https://grp2winereviews.s3.amazonaws.com/wine_data_cleaned.csv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
df.show()

+------+-----+----------+------------------+-------------------+----+
|points|price|  province|           variety|             winery|Year|
+------+-----+----------+------------------+-------------------+----+
|    87| 14.0|    Oregon|        Pinot Gris|          Rainstorm|2013|
|    87| 65.0|    Oregon|        Pinot Noir|       Sweet Cheeks|2012|
|    87| 24.0|    Alsace|   GewÃ¼rztraminer|           Trimbach|2012|
|    87| 27.0|    Alsace|        Pinot Gris| Jean-Baptiste Adam|2012|
|    87| 19.0|California|Cabernet Sauvignon| Kirkland Signature|2011|
|    87| 30.0|    Alsace|   GewÃ¼rztraminer|         Leon Beyer|2012|
|    87| 34.0|California|Cabernet Sauvignon|   Louis M. Martini|2012|
|    87| 12.0|California|        Chardonnay|           Mirassou|2012|
|    85| 20.0|California|        Chardonnay|           Ca' Momi|2015|
|    87| 20.0|    Oregon|        Pinot Noir|            Acrobat|2013|
|    87| 19.0|   Sicilia|       White Blend| Baglio di Pianetto|2007|
|    87| 22.0|Califo

In [ ]:
# Configure settings for RDS - Group2Wineos password isn't an important password and was kept simple for this exercise.
mode = "append"
jdbc_url="jdbc:postgresql://grp2rdsinstance.cwkbfcctxu7y.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"root", 
          "password": "Group2Wineos", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write df to table in RDS
df.write.jdbc(url=jdbc_url, table='wine_data', mode=mode, properties=config)